# SETUP

In [12]:
from requests import get
import requests
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
from IPython.display import HTML
import pyodbc


from pandas.io.json import json_normalize

In [13]:
from mycbr_py_api import MyCBRRestApi as mycbr

In [17]:

casebase=list()

def retrieve_c_att22(case,k):
    global casebase
    cases=casebase
    cases=cases[cases["IndustrySubgroupCode"].astype(float)==float(case["IndustrySubgroupCode"])]
    cases=cases[cases["MunicipalityNumber"].astype(int)==int(case["MunicipalityNumber"])]
    results = cases.apply(pd.to_numeric, errors='coerce').fillna(cases).sort_values(by='similarity', ascending=False)
    results['similarity'] = results['similarity'].astype(float)
    results['NonCompliance'] = results['NonCompliance'].astype(int)
    #returned_df = results.apply(pd.to_numeric, errors='coerce').fillna(results)
    #results=results.head(k)
    return results


In [18]:
casebase=list()
initCBCopy=casebase
currentCBCopy=casebase

#Add multiple items to the case base
def addManyCases(cases):
    cases=cases.to_dict('records')
    rec={'cases':cases}
    #print(rec)
    #structure of case dictionary must be {'cases':[{'IndustrySubgroupCode':45.2},{"Municipality":"Amik"}]}
    #NB! int values must be formatted as strings
    results=requests.post(url=base_url + 'concepts/KP/casebases/CB_csvImport/cases', json=rec,headers={'Accept': 'application/json'})
    return results
#Fill casebase in case its empty
def Getallcases():
    global casebase
    # Specifying the ODBC driver, server name, database, etc. directly
    cnxn = pyodbc.connect('DRIVER={SQL Server Native Client 11.0};SERVER=localhost;DATABASE=IJCAI2022;Trusted_Connection=yes;')
    cursor = cnxn.cursor()
    traindateid=""
    testdateid=""
    cases=pd.read_sql("SELECT  [MuncipalityIndex] as MunicipalityIndex ,[IndustrySubgroupCodeIndex],[IndustryGroupCode],[IndustryAreaCode],[IndustryCode],[Fylke],[MunicipalityNumber],  [InspectionId] as 'InspectionId' ,[IndustryMainAreaCode],ControlPointText,[IndustrySubgroupCode],[Municipality],[NonCompliance]"
+",round(isNull(SannsynlighetForBruddGittKPogKommune,(1.0/(6.0+isNull(tttg2,0))))*100,0) as ProbabilityForNonComplianceGivenControlPointandMunicipality"
+",round(isNull(SannsynlighetForBruddGittKPogNaring,(1.0/(6.0+isNull(tttg,0))))*100,0) as ProbabilityForNonComplianceGivenControlPointandIndustrySubgroup"
+",round(isNull(AntBruddGittKPogNaring,0),0) as NumberOfNonComplianceGivenControlPointandIndustrySubgroup"
+",round(isNull(AntBruddGittKPogKommune,0),0) as NumberOfNonComplianceGivenControlPointandMunicipality"
                      +" FROM [IJCAI2022].[dbo].[BayesianDynamicChecklistLocalDb]"
+" Left join(Select ( (Convert(real,isNull(Count(IndustrySubgroupCode),0))+1.0)/(AVG(Convert(real, Kommune3))+6.0))  as SannsynlighetForBruddGittKPogNaring,Count(IndustrySubgroupCode) as AntBruddGittKPogNaring,ControlPointText as KPT2, IndustrySubgroupCode as Nugk"
+" FROM [IJCAI2022].[dbo].[BayesianDynamicChecklistLocalDb]"  
+" join (Select Count(IndustrySubgroupCode) as Kommune3 , ControlPointText as KT2, IndustrySubgroupCode as Kom"
+" FROM [IJCAI2022].[dbo].[BayesianDynamicChecklistLocalDb]"+traindateid+"  group by ControlPointText, IndustrySubgroupCode) as a on a.Kom=IndustrySubgroupCode and ControlPointText=KT2" 
+" join (Select Count(Municipality) as Kommune5 ,  max(NonCompliance) as AntB2"
+" FROM [IJCAI2022].[dbo].[BayesianDynamicChecklistLocalDb]" +traindateid+") as c on c.AntB2=NonCompliance"
                    +traindateid
+" group by ControlPointText, NonCompliance, IndustrySubgroupCode) as b on b.KPT2=ControlPointText and b.Nugk=IndustrySubgroupCode"
+" Left join(Select ( (Convert(real,isNull(Count(Municipality),0))+1.0)/(AVG(Convert(real, Kommune3))+6.0))  as SannsynlighetForBruddGittKPogKommune,Count(Municipality) as AntBruddGittKPogKommune, ControlPointText as KPT, Municipality as KOMM"
+" FROM [IJCAI2022].[dbo].[BayesianDynamicChecklistLocalDb] join (Select Count(Municipality) as Kommune3 , ControlPointText as KT2, Municipality as Kom"
+" FROM [IJCAI2022].[dbo].[BayesianDynamicChecklistLocalDb]"+traindateid+" group by ControlPointText, Municipality) as a on a.Kom=Municipality and ControlPointText=KT2" 
+" join (Select Count(Municipality) as Kommune5 ,  max(NonCompliance) as AntB2"
+" FROM [IJCAI2022].[dbo].[BayesianDynamicChecklistLocalDb]" +traindateid+") as c on c.AntB2=NonCompliance"
                    +traindateid+"  group by ControlPointText, NonCompliance, Municipality) as a on a.KOMM=Municipality and a.KPT=ControlPointText"
+" left join(Select Count(IndustrySubgroupCode) as tttg , ControlPointText as KT10, IndustrySubgroupCode as Kom10"
+" FROM [IJCAI2022].[dbo].[BayesianDynamicChecklistLocalDb]"+traindateid+" group by ControlPointText, IndustrySubgroupCode) as z on z.Kom10=IndustrySubgroupCode and ControlPointText=z.KT10" 
+" left join(Select Count(Municipality) as tttg2 , ControlPointText as KT10, Municipality as NUG10"
+" FROM [IJCAI2022].[dbo].[BayesianDynamicChecklistLocalDb]"+traindateid+" group by ControlPointText, Municipality) as zz on ControlPointText=zz.KT10 and zz.NUG10=Municipality" 
+testdateid
                      +" Order by InspectionDateId",cnxn)

    cases['NonCompliance']= cases['NonCompliance'].map(str)
    cases['NumberOfNonComplianceGivenControlPointandIndustrySubgroup']= cases['NumberOfNonComplianceGivenControlPointandIndustrySubgroup'].map(str)
    cases['NumberOfNonComplianceGivenControlPointandMunicipality']= cases['NumberOfNonComplianceGivenControlPointandMunicipality'].map(str)
    cases['ProbabilityForNonComplianceGivenControlPointandIndustrySubgroup']= cases['ProbabilityForNonComplianceGivenControlPointandIndustrySubgroup'].astype(int)
    cases['ProbabilityForNonComplianceGivenControlPointandMunicipality']= cases['ProbabilityForNonComplianceGivenControlPointandMunicipality'].astype(int)
    cases['ProbabilityForNonComplianceGivenControlPointandIndustrySubgroup']= cases['ProbabilityForNonComplianceGivenControlPointandIndustrySubgroup'].map(str)
    cases['ProbabilityForNonComplianceGivenControlPointandMunicipality']= cases['ProbabilityForNonComplianceGivenControlPointandMunicipality'].map(str)
    cases['IndustrySubgroupCodeIndex']= cases['IndustrySubgroupCodeIndex'].map(str)
    cases['MunicipalityIndex']= cases['MunicipalityIndex'].map(str)
    cases['InspectionId']=cases['InspectionId'].apply(int)+0
    cases['InspectionId']= cases['InspectionId'].map(str)
    cases['MunicipalityNumber']= cases['MunicipalityNumber'].map(str)
    cases['IndustryAreaCode']= cases['IndustryAreaCode'].map(str)
    cases['IndustryCode']= cases['IndustryCode'].map(str)
    cases['IndustryCode']=cases['IndustryCode'].replace(to_replace=r',', value='.', regex=True)
    cases=cases.assign(similarity=(cases['ProbabilityForNonComplianceGivenControlPointandIndustrySubgroup'].astype(float)+cases['ProbabilityForNonComplianceGivenControlPointandMunicipality'].astype(float))/2)
    cases=cases.dropna()
    casebase=cases

def fillCaseBase(splitindex):
    global casebase
    # Specifying the ODBC driver, server name, database, etc. directly
    cnxn = pyodbc.connect('DRIVER={SQL Server Native Client 11.0};SERVER=localhost;DATABASE=IJCAI2022;Trusted_Connection=yes;')
    cursor = cnxn.cursor()

    traindateid=""
    testdateid=""
    cases=pd.read_sql("SELECT  [MuncipalityIndex] as MunicipalityIndex ,[IndustrySubgroupCodeIndex],[IndustryGroupCode],[IndustryAreaCode],[IndustryCode],[Fylke],[MunicipalityNumber],  [InspectionId] as 'InspectionId' ,[IndustryMainAreaCode],ControlPointText,[IndustrySubgroupCode], 'None' as UnderThemeName,[Municipality],[NonCompliance]"
+",round(isNull(SannsynlighetForBruddGittKPogKommune,(1.0/(6.0+isNull(tttg2,0))))*100,0) as ProbabilityForNonComplianceGivenControlPointandMunicipality"
+",round(isNull(SannsynlighetForBruddGittKPogNaring,(1.0/(6.0+isNull(tttg,0))))*100,0) as ProbabilityForNonComplianceGivenControlPointandIndustrySubgroup"
+",round(isNull(AntBruddGittKPogNaring,0),0) as NumberOfNonComplianceGivenControlPointandIndustrySubgroup"
+",round(isNull(AntBruddGittKPogKommune,0),0) as NumberOfNonComplianceGivenControlPointandMunicipality"
                      +" FROM [IJCAI2022].[dbo].[BayesianDynamicChecklistLocalDbRandomizedTest"+str(splitindex)+"]"
+" Left join(Select ( (Convert(real,isNull(Count(IndustrySubgroupCode),0))+1.0)/(AVG(Convert(real, Kommune3))+6.0))  as SannsynlighetForBruddGittKPogNaring,Count(IndustrySubgroupCode) as AntBruddGittKPogNaring,ControlPointText as KPT2, IndustrySubgroupCode as Nugk"
+" FROM [IJCAI2022].[dbo].[BayesianDynamicChecklistLocalDbRandomizedTrain"+str(splitindex)+"]"+" "  
+" join (Select Count(IndustrySubgroupCode) as Kommune3 , ControlPointText as KT2, IndustrySubgroupCode as Kom"
+" FROM [IJCAI2022].[dbo].[BayesianDynamicChecklistLocalDbRandomizedTrain"+str(splitindex)+"]"+" "+traindateid+"  group by ControlPointText, IndustrySubgroupCode) as a on a.Kom=IndustrySubgroupCode and ControlPointText=KT2" 
+" join (Select Count(Municipality) as Kommune5 ,  max(NonCompliance) as AntB2"
+" FROM [IJCAI2022].[dbo].[BayesianDynamicChecklistLocalDbRandomizedTrain"+str(splitindex)+"]"+" " +traindateid+") as c on c.AntB2=NonCompliance"
                    +traindateid
+" group by ControlPointText, NonCompliance, IndustrySubgroupCode) as b on b.KPT2=ControlPointText and b.Nugk=IndustrySubgroupCode"
+" Left join(Select ( (Convert(real,isNull(Count(Municipality),0))+1.0)/(AVG(Convert(real, Kommune3))+6.0))  as SannsynlighetForBruddGittKPogKommune,Count(Municipality) as AntBruddGittKPogKommune, ControlPointText as KPT, Municipality as KOMM"
+" FROM [IJCAI2022].[dbo].[BayesianDynamicChecklistLocalDbRandomizedTrain"+str(splitindex)+"]"+"  join (Select Count(Municipality) as Kommune3 , ControlPointText as KT2, Municipality as Kom"
+" FROM [IJCAI2022].[dbo].[BayesianDynamicChecklistLocalDbRandomizedTrain"+str(splitindex)+"]"+" "+traindateid+" group by ControlPointText, Municipality) as a on a.Kom=Municipality and ControlPointText=KT2" 
+" join (Select Count(Municipality) as Kommune5 ,  max(NonCompliance) as AntB2"
+" FROM [IJCAI2022].[dbo].[BayesianDynamicChecklistLocalDbRandomizedTrain"+str(splitindex)+"]"+" " +traindateid+") as c on c.AntB2=NonCompliance"
                    +traindateid+"  group by ControlPointText, NonCompliance, Municipality) as a on a.KOMM=Municipality and a.KPT=ControlPointText"
+" left join(Select Count(IndustrySubgroupCode) as tttg , ControlPointText as KT10, IndustrySubgroupCode as Kom10"
+" FROM [IJCAI2022].[dbo].[BayesianDynamicChecklistLocalDbRandomizedTrain"+str(splitindex)+"]"+" "+traindateid+" group by ControlPointText, IndustrySubgroupCode) as z on z.Kom10=IndustrySubgroupCode and ControlPointText=z.KT10" 
+" left join(Select Count(Municipality) as tttg2 , ControlPointText as KT10, Municipality as NUG10"
+" FROM [IJCAI2022].[dbo].[BayesianDynamicChecklistLocalDbRandomizedTrain"+str(splitindex)+"]"+" "+traindateid+" group by ControlPointText, Municipality) as zz on ControlPointText=zz.KT10 and zz.NUG10=Municipality" 
+" where InspectionDateId<20190601"
                      +" Order by InspectionDateId " + testdateid,cnxn)
    #print(cases)
    cases['NonCompliance']= cases['NonCompliance'].map(str)
    cases['NumberOfNonComplianceGivenControlPointandIndustrySubgroup']= cases['NumberOfNonComplianceGivenControlPointandIndustrySubgroup'].map(str)
    cases['NumberOfNonComplianceGivenControlPointandMunicipality']= cases['NumberOfNonComplianceGivenControlPointandMunicipality'].map(str)
    cases['ProbabilityForNonComplianceGivenControlPointandIndustrySubgroup']= cases['ProbabilityForNonComplianceGivenControlPointandIndustrySubgroup'].astype(int)
    cases['ProbabilityForNonComplianceGivenControlPointandMunicipality']= cases['ProbabilityForNonComplianceGivenControlPointandMunicipality'].astype(int)
    cases['ProbabilityForNonComplianceGivenControlPointandIndustrySubgroup']= cases['ProbabilityForNonComplianceGivenControlPointandIndustrySubgroup'].map(str)
    cases['ProbabilityForNonComplianceGivenControlPointandMunicipality']= cases['ProbabilityForNonComplianceGivenControlPointandMunicipality'].map(str)
    cases['IndustrySubgroupCodeIndex']= cases['IndustrySubgroupCodeIndex'].map(str)
    cases['MunicipalityIndex']= cases['MunicipalityIndex'].map(str)
    cases['InspectionId']=cases['InspectionId'].apply(int)+0
    cases['InspectionId']= cases['InspectionId'].map(str)
    cases['MunicipalityNumber']= cases['MunicipalityNumber'].map(str)
    cases['IndustryAreaCode']= cases['IndustryAreaCode'].map(str)
    cases['IndustryCode']= cases['IndustryCode'].map(str)
    cases['IndustryCode']=cases['IndustryCode'].replace(to_replace=r',', value='.', regex=True)
    cases=cases.assign(similarity=(cases['ProbabilityForNonComplianceGivenControlPointandIndustrySubgroup'].astype(float)+cases['ProbabilityForNonComplianceGivenControlPointandMunicipality'].astype(float))/2)
    cases=cases.dropna()
    casebase=cases
#Delete all items from the case base
def deleteAllCases():
    iaat=0


#Updates casebase with the current set of cases (currentCBCopy)
def updateCaseBaseProb(cases):
    deleteAllCases()
    cases['NonCompliance']= cases['NonCompliance'].map(str)
    cases['NumberOfNonComplianceGivenControlPointandIndustrySubgroup']= cases['NumberOfNonComplianceGivenControlPointandIndustrySubgroup'].map(str)
    cases['NumberOfNonComplianceGivenControlPointandMunicipality']= cases['NumberOfNonComplianceGivenControlPointandMunicipality'].map(str)
    cases['ProbabilityForNonComplianceGivenControlPointandIndustrySubgroup']= cases['ProbabilityForNonComplianceGivenControlPointandIndustrySubgroup'].map(str)
    cases['ProbabilityForNonComplianceGivenControlPointandMunicipality']= cases['ProbabilityForNonComplianceGivenControlPointandMunicipality'].map(str)
    cases['IndustrySubgroupCodeIndex']= cases['IndustrySubgroupCodeIndex'].map(str)
    cases['MunicipalityIndex']= cases['MunicipalityIndex'].map(str)
    cases['InspectionId']=cases['InspectionId'].apply(int)+0
    cases['InspectionId']= cases['InspectionId'].map(str)
    cases['MunicipalityNumber']= cases['MunicipalityNumber'].map(str)
    cases['IndustryAreaCode']= cases['IndustryAreaCode'].map(str)
    cases['IndustryCode']= cases['IndustryCode'].map(str)
    cases['IndustryCode']=cases['IndustryCode'].replace(to_replace=r',', value='.', regex=True)

    cases=cases.drop(columns=[ 'caseID'])
    cases=cases.dropna()
    res=addManyCases(cases)
    return res;



#Restores the casebase back to it's original form
def restoreOriginalCaseBase(splitindex):
    deleteAllCases()
    fillCaseBase(splitindex)
    #return res;


# EXPERIMENT, STATISTICS FOR GENERATED CHECKLISTS (NBI)

In [21]:
#Control point is synonymous with an item
import time
initCBCopy=list();
currentCBCopy=initCBCopy
traintime=0
timetotal=0
precctot=0
accctot=0
preccgttot=0
recctot=0
fh2=open("Log_" + "NBIChecklistsTest" + '.txt', 'w+')
for ik in range(0,8):
    
    start_time = time.time()
    if len(initCBCopy)==0:
        fillCaseBase(ik)
        initCBCopy=casebase
        currentCBCopy=initCBCopy
    else:
        restoreOriginalCaseBase(ik)
        initCBCopy=casebase
        currentCBCopy=initCBCopy
    traintime=traintime+(time.time()-start_time)
    print("Train time: "+str(traintime/(ik+1)))
    fh2.write("Train time: "+str(traintime/(ik+1)))
    noncompliance=0 #true positive(gt)
    controlpoints=0 #predicted positive(gt)
    noncompliancengt=0 #true positive (ngt)
    controlpointsngt=0#predicted positive (ngt)
    truepositiveval=0
    truepositive=0
    falsepositivegt=0
    falsepositivengt=0
    precisionval=0
    recallval=0
    accuracyval=0
    precision=0
    precisiongt=0
    lengthprecgt=0
    recall=0
    accuracy=0
    lengthprec=0.00001
    lengthvalprec=0
    lengthrec=0.00001
    lengthvalrec=0
    lengthacc=0.00001
    lengthvalacc=0
    truenegative=0
    falsenegative=0
    similarity=0
    counter=0
    Kcp=15
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', -1)
    noncompliantInspection=0
    numberofOverlappingControlpoints=0
    inspections=casebase.drop_duplicates(subset = ["InspectionId"])
    cases=casebase
    cases["NonCompliance"]=cases["NonCompliance"].astype(int)
    casestest=len(cases.drop_duplicates(subset = ["ControlPointText"]))
    cpCases=list()
    negatives=list()
    cltotallen=0
    inspectionstot=0
    precchecklists=0
    recchecklists=0
    accchecklists=0
    precchecklistsval=0
    recchecklistsval=0
    accchecklistsval=0
    print(casestest)
    for ind, case in inspections.iterrows():
   
        inspectionstot+=1
        uniquechecklistlengthval=0
        cpCases=list()
        counter+=1
        if counter>500:
            counter=0
            print(ind)
            
        r=0#Stop variable to ensure at least 10 cases are selected for each inspection (case)
        k=500#Finding the Top K cases
        existingChecklist=cases[cases['InspectionId']==case['InspectionId']]#Retrieve the existing checklist
        clck=cases[cases["IndustrySubgroupCode"].astype(float)==float(case["IndustrySubgroupCode"])]
        clck=clck[clck["MunicipalityNumber"].astype(int)==int(case["MunicipalityNumber"])]
        clck["NonCompliance"]=clck["NonCompliance"].astype(float)
        negatives=clck
        if len(clck)>0:
            caseAvgNoncompliancePrControlpoint=clck.groupby(["ControlPointText"],as_index=False).mean()

            #print(caseAvgNoncompliancePrControlpoint)
        while r==0:
            checklist=retrieve_c_att22(case,k)#Retrieve items that matches each inspection (input case). checklist=CL from paper


            #Find the mean value of each column in the returned data set of items. 

            uniqueChecklist=checklist.drop_duplicates(subset = ["ControlPointText"])#Find all unique items by removing duplicates
            uniqueChecklist=uniqueChecklist.head(Kcp)
            if len(uniqueChecklist)>(0):#Check if there are at least 10 unique items
                truepositive=0
                truepositiveval=0
                r=1#Stops the while loop since 10 unique items are found
                #controlpoints+=len(uniqueChecklist) 
                #Record statistics. Add to the total, the number of unique items that were found in current generated checklist.  
                similarity+=uniqueChecklist["similarity"].sum() 
                #Record statistics. Add the ratio/fraction of non-compliance for each of the (unique) item in the checklist.

                precpercl=0
                lengthorgcl=0
                for ind2, generatedChecklistControlpoint in uniqueChecklist.iterrows():#Find overlap between the existing and new generated checklist
                    cltotallen+=1
                    negatives=negatives[negatives["ControlPointText"]!=generatedChecklistControlpoint["ControlPointText"]]#This is wrong. Negatives need to be outside the whole checklist
                    excp=existingChecklist[existingChecklist["ControlPointText"]==generatedChecklistControlpoint["ControlPointText"]]
                    if len(excp)>0:
                        numberofOverlappingControlpoints+=1 
                        precpercl+=((excp["NonCompliance"].sum())/len(excp["NonCompliance"]))
                        lengthorgcl+=1
                        controlpoints+=1#matchlen
                        

                    caseAvgNoncompliancePrControlpoint2=caseAvgNoncompliancePrControlpoint[caseAvgNoncompliancePrControlpoint["ControlPointText"]==generatedChecklistControlpoint["ControlPointText"]]

                    summ2=caseAvgNoncompliancePrControlpoint2["NonCompliance"].sum() 
                    if len(caseAvgNoncompliancePrControlpoint2["NonCompliance"])>0:
                        noncompliance+=(summ2/len(caseAvgNoncompliancePrControlpoint2["NonCompliance"]))
                        truepositiveval+=(summ2/len(caseAvgNoncompliancePrControlpoint2["NonCompliance"]))
                    
                    matchlen=len(caseAvgNoncompliancePrControlpoint2["NonCompliance"])
                    if matchlen>1:
                        print(caseAvgNoncompliancePrControlpoint2["NonCompliance"])
                    controlpointsngt+=1
                    falsepositivegt+=(matchlen-summ2)
                    if matchlen>0:
                        precisionval+=(summ2/matchlen)
                        lengthvalprec+=1
                        uniquechecklistlengthval+=1
                    summ3=0
                    if matchlen==0:#if validation record of checklist item does not exist
                        cpCases=cases[cases["IndustrySubgroupCode"].astype(float)==float(generatedChecklistControlpoint["IndustrySubgroupCode"])]
                        cpCases=cpCases[cpCases["MunicipalityNumber"].astype(int)==int(generatedChecklistControlpoint["MunicipalityNumber"])]
                        cpCases=cpCases[cpCases["ControlPointText"]==generatedChecklistControlpoint["ControlPointText"]]
                        cpCases["NonCompliance"]=cpCases["NonCompliance"].astype(float)
                        summ3=cpCases["NonCompliance"].sum()
                        
                        
                        falsepositivengt+=(len(cpCases["NonCompliance"])-summ3)
                        if len(cpCases["NonCompliance"])>0:
                            noncompliancengt+=(summ3/len(cpCases["NonCompliance"]))
                            truepositive+=(summ3/len(cpCases["NonCompliance"]))
                            precision+=(summ3/len(cpCases["NonCompliance"]))
                            lengthprec+=1
                        
                if lengthorgcl>0:
                    precisiongt+=(precpercl/lengthorgcl)
                    lengthprecgt+=1      
                    
                #Calculate statistics on checklists level
                uniquenegatives=len(negatives.drop_duplicates(subset = ["ControlPointText"]))
                negativecpy=negatives.copy()
                negativecpy["NonCompliance"]=negativecpy["NonCompliance"].astype(float)
                groupedby=negativecpy.groupby(["ControlPointText"],as_index=False).count()
                groupedbys=negativecpy.groupby(["ControlPointText"],as_index=False).sum()
                groupedby["NonCompliance"]=groupedbys["NonCompliance"]/groupedby["NonCompliance"]
                uniquecalcnegatives=groupedby.drop_duplicates(subset = ["ControlPointText"])
                noncompliancenegatives=uniquecalcnegatives["NonCompliance"].sum()

                falsenegativeelement=0
                if uniquenegatives>0:
                    falsenegativeelement=noncompliancenegatives/uniquenegatives #To avoid selection bias effects
                truenegativeelement=1-falsenegativeelement
                
                truepositivesprchecklistval=(truepositiveval)
                falsepositivesprchecklistval=(uniquechecklistlengthval-truepositivesprchecklistval)
                
                truepositivesprchecklist=(truepositive+truepositiveval)
                falsepositivesprchecklist=(len(uniqueChecklist)-(truepositive+truepositiveval))
                truenegativesprchecklist=truenegativeelement*uniquenegatives
                falsenegativesprchecklist=falsenegativeelement*uniquenegatives
                #len(uniqueChecklist)+uniquenegatives=len(unique items)
                
                precprchecklist=truepositivesprchecklist/len(uniqueChecklist)
                precchecklists+=precprchecklist
                #print("prec: "+str(precprchecklist))
                precprchecklistval=0
                if uniquechecklistlengthval>0:
                    precprchecklistval=truepositivesprchecklistval/uniquechecklistlengthval
                precchecklistsval+=precprchecklistval
                #print("prec(val): "+str(precprchecklistval))
                recprchecklist=0
                if (truepositivesprchecklist+falsenegativesprchecklist)>0:
                    recprchecklist=truepositivesprchecklist/(truepositivesprchecklist+falsenegativesprchecklist)
                recchecklists+=recprchecklist
               # print("rec: "+str(recprchecklist))
                recprchecklistval=0
                if (truepositivesprchecklistval+falsenegativesprchecklist)>0:
                    recprchecklistval=truepositivesprchecklistval/(truepositivesprchecklistval+falsenegativesprchecklist)
                recchecklistsval+=recprchecklistval
                #print("rec(val): "+str(recprchecklistval))
                accprchecklist=0
                if (len(uniqueChecklist)+uniquenegatives)>0:
                    accprchecklist=(truepositivesprchecklist+truenegativesprchecklist)/(len(uniqueChecklist)+uniquenegatives)
                accchecklists+=accprchecklist
                #print("acc: "+str(accprchecklist))
                accprchecklistval=0
                if (uniquechecklistlengthval+uniquenegatives)>0:
                    accprchecklistval=(truepositivesprchecklistval+truenegativesprchecklist)/(uniquechecklistlengthval+uniquenegatives)
                accchecklistsval+=accprchecklistval
                #print("acc(val): "+str(accprchecklistval))
                
                
                #|true positives for each checklist|=sum(true positive elements in checklist)
                #|false positives for each checklist|=sum(false positive elements in checklist)
                #|true negatives for each checklist|=|true negative element|*|unique items(2)|
                #|false negatives for each checklist|=|false negative element|*|unique items(2)|
                #prec pr checklist
                #rec pr checklist
                #acc pr checklist
                        
            k+=500 #Increment k while there are less then 10 unique items
    if inspectionstot>0:
        precctot+=precchecklists/inspectionstot
        accctot+=accchecklists/inspectionstot
        preccgttot+=precisiongt/lengthprecgt
        recctot+=recchecklists/inspectionstot        

    timetotal=timetotal+(time.time()-start_time)
    print("Results for current validation fold: "+ str(ik))
    print("Current avg time:"+ str(timetotal/(ik+1)))
    fh2.write("\nCurrent avg time:"+ str(timetotal/(ik+1)))
    print("Precision (gt): "+str(precisiongt/lengthprecgt))
    fh2.write("\nPrecision (gt): "+str(precisiongt/lengthprecgt))
    print("Precision: "+str((precchecklists)/(inspectionstot)))
    fh2.write("\nPrecision: "+str((precchecklists)/(inspectionstot)))
    print("Recall: "+str((recchecklists)/(inspectionstot)))
    fh2.write("\nRecall: "+str((recchecklists)/(inspectionstot)))
    print("Accuracy: "+str((accchecklists)/(inspectionstot)))
    fh2.write("\nAccuracy: "+str((accchecklists)/(inspectionstot)))
    print("Average number of items per checklist(gt): "+str(controlpoints/inspectionstot))
    fh2.write("\nAverage number of items per checklist(gt): "+str(controlpoints/inspectionstot))
    print("Average number of items per checklist: "+str((controlpointsngt)/inspectionstot))
    fh2.write("\nAverage number of items per checklist: "+str((controlpointsngt)/inspectionstot))
    print("Average number of non-compliant items per checklist(gt): "+str(precisiongt/inspectionstot))
    fh2.write("\nAverage number of non-compliant items per checklist(gt): "+str(precisiongt/inspectionstot))
    print("Average number of non-compliant items per checklist: "+str((noncompliance+noncompliancengt)/inspectionstot))
    fh2.write("\nAverage number of non-compliant items per checklist: "+str((noncompliance+noncompliancengt)/inspectionstot))

    print("Average similarity: "+str(similarity/(lengthprecgt+lengthvalprec)))
    fh2.write("\nAverage similarity: "+str(similarity/(lengthprecgt+lengthvalprec)))


print("\nTotal Average Accuracy: "+str(accctot/8))
print("\nTotal Average Prec: "+str(precctot/8))
print("\nTotal Average Rec: "+str(recctot/8))
print("\nTotal Average Precgt: "+str(preccgttot/8))                             
fh2.write("\nAverage Accuracy: "+str(accctot/8))
fh2.write("\nAverage Prec: "+str(precctot/8))
fh2.write("\nAverage Rec: "+str(recctot/8))
fh2.write("\nAverage Precgt: "+str(preccgttot/8))
fh2.close()

Train time: 7.349400520324707
947


<ipython-input-21-3a1e6ba26f27>:55: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


8768
17637
27768
37690
47005
57223
67040
76364
86097
93934
102596
110716
118770
125946
133702
Results for current validation fold: 0
Current avg time:575.1094858646393
Precision (gt): 0.2545070896335419
Precision: 0.28559012864666294
Recall: 0.6961871242083736
Accuracy: 0.5126893897738318
Average number of items per checklist(gt): 9.613167636171339
Average number of items per checklist: 14.107086197778953
Average number of non-compliant items per checklist(gt): 0.2434708223940123
Average number of non-compliant items per checklist: 4.08938908490317
Average similarity: 23.70525355005178
Train time: 7.259006857872009
956


<ipython-input-21-3a1e6ba26f27>:55: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


8650
17561


KeyboardInterrupt: 